In [1]:
import pandas as pd 
import numpy as np 
import statsmodels.api as sm 
from statsmodels.regression.linear_model import OLS 
# import matplotlib.pyplot as plt 

df = pd.read_csv("IVE_tickbidask.txt", header = None, names = ['date','time','price','bid','ask','volume'])[:100]
#required data: time, bid, bid size, ask, ask size

In [2]:
def format_data(df: pd.DataFrame)->pd.DataFrame:
    df['datetime'] = df[['date','time']].apply(lambda x: ' '.join(x),axis = 1)
    df['datetime'] = pd.to_datetime(df['datetime'],infer_datetime_format = True )
    df = df[['datetime','price','bid','ask','volume']]
    return df

In [3]:
df = format_data(df)

# OrderFlow imbalance (OFI) vs Trade Flow Imbalance (TFI)<br>
## These measures determine the supply and demand on the orderbook<br>
-  OFI: Measures the orders that were executed
- TFI: Measures any changes on the order book<br>


# OrderFlow:

In [4]:
def ofi(df:pd.DataFrame)-> pd.DataFrame:
    df['mid-price-change'] = ((df['bid'] - df['ask'])/2).diff().div(0.1)
    df['prev-bid'] = df['bid'].shift()
    df['prev-bid-size'] = df['bid-size'].shift()
    df['prev-ask'] = df['ask'].shift()
    df['prev-ask-size'] = df['ask-size'].shift()
    df.dropna(inplace =True)

    bid_geq = df['bid'] >= df['prev-bid']
    bid_leq = df['bid'] <= df['prev-bid']
    ask_geq = df['ask'] >= df['prev-ask']
    ask_leq = df['ask'] <= df['prev-ask']

    df['OFI'] = pd.Series(np.zeros(len(df)))
    df['OFI'].loc[bid_geq] += df['bid-size'].loc[bid_geq]
    df['OFI'].loc[bid_leq] += df['prev-bid-size'].loc[bid_leq]
    df['OFI'].loc[ask_geq] += df['prev-ask-size'].loc[ask_geq]
    df['OFI'].loc[ask_leq] += df['ask-size'].loc[ask_leq]

    df = df[['mid-price change','OFI']].resample().sum().dropna() 
    return df


In [ ]:
def plot_ofi_and_ols_summary(df: pd.DataFrame)->pd.DataFrame:
    df.plot(kind='scatter', grid=True, 
            x='OFI', y='Mid-Price Change', 
            title = 'OFI'
            alpha=0.5, figsize=(12,10))
    ofi_ = sm.add_constant(df['OFI'])
    ols = OLS(df['mid-price-change'], ofi_).fit()
    print(ols.summary2())